In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |█████████████████████████████▏  | 283.2 MB 84.7 MB/s eta 0:00:012

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 200 kB 63.3 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317143 sha256=85101d804b8ea76270af53e7870db8814059cdb4d22b56b4ce500bcc648d64ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-e6v66pzh/wheels/27/3e/a7/888155c6a7f230b13a394f4999b90fdfaed00596c68d3de307
Successfully built pyspark
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import os, re

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/13 17:43:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
def read_lines(path = '../datasets/iam'):

    with open(os.path.join(path, 'lines.txt'), 'r') as f:
        lines = f.readlines()

    description = lines[:23]
    data = lines[23:]

    rows = []
    for line in data:
        temp = line.split()
        row = temp[:8]
        row.extend([' '.join(temp[8:])])
        rows.append(row)

    cols = ['id', 'result', 'graylevel', 'components', 'x', 'y', 'w', 'h', 'message']
    
    return rows, cols
    
def clean_message(m):
    m = m.replace('|-|', ' - ')
    m = m.replace('|*|', ' * ')
    temp0 = m.replace('.|"', '"|.')
    temp1 = re.sub(r'"\|([\w\d\s\|-]+)\|"', r'"\1"', temp0)
    temp2 = re.sub(r'([^\'\(])\|([\w\d\"\#\(])', r'\1 \2', temp1)
    temp3 = re.sub(r'([^\'\(\"])\|([\w\d\"\#\(])', r'\1 \2', temp2)
    temp4 = re.sub(r'^"\s', r'"', temp3)
    return temp4.replace('|', '').replace(' ". ', '". ')

In [ ]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()


path = '../datasets/iam'

rows, cols = read_lines()

df = spark.createDataFrame(rows, schema = cols)

for c in ['components', 'graylevel', 'x', 'y', 'w', 'h']:
    df = df.withColumn(c, df[c].cast(IntegerType()))

clean_message_udf = udf(lambda m: clean_message(m), StringType())

df = df.withColumn("clean_message", clean_message_udf(col("message")))

In [7]:
df.show(5)

+-----------+------+---------+----------+---+----+----+---+--------------------+--------------------+
|         id|result|graylevel|components|  x|   y|   w|  h|             message|       clean_message|
+-----------+------+---------+----------+---+----+----+---+--------------------+--------------------+
|a01-000u-00|    ok|      154|        19|408| 746|1661| 89|A|MOVE|to|stop|Mr...|A MOVE to stop Mr...|
|a01-000u-01|    ok|      156|        19|395| 932|1850|105|nominating|any|mo...|nominating any mo...|
|a01-000u-02|    ok|      157|        16|408|1106|1986|105|is|to|be|made|at|...|is to be made at ...|
|a01-000u-03|   err|      156|        23|430|1290|1883| 70|M Ps|tomorrow|.|M...|M Ps tomorrow. Mr...|
|a01-000u-04|    ok|      157|        20|395|1474|1830| 94|put|down|a|resolu...|put down a resolu...|
+-----------+------+---------+----------+---+----+----+---+--------------------+--------------------+
only showing top 5 rows

